In [2]:
!pip install torch-fidelity
!pip install torchmetrics[image]

  Using cached torch-2.3.0-cp39-cp39-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.

In [3]:
generate_images_folder = './e2e_stg3_64/96/images'
orig_images_folder = '/root/workspace/sharing/lgp-mlu-dev/share/celaba64/50k/'

gen_image_prefix = 'output_'
orig_images_prefix = ''

In [4]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torchmetrics.image.fid import FrechetInceptionDistance


# 이미지 불러오기 및 텐서로 변환하는 함수
def load_images_from_folder(folder, start_dir):
    images = []
    transform = transforms.Compose([
        transforms.Resize((64, 64)),  # 이미지 크기를 128x128로 조정
        transforms.ToTensor(),
        transforms.Lambda(lambda x: (x * 255).byte())
    ])
    for filename in os.listdir(folder):
        if filename.startswith(start_dir):
            img_path = os.path.join(folder, filename)
            
            if not os.path.isfile(img_path):
                    continue
                    
            with Image.open(img_path) as img:                    
                img = transform(img)
                images.append(img) 
    return torch.stack(images)

# vae_로 시작하는 이미지 불러오기
vae_images = load_images_from_folder(generate_images_folder, gen_image_prefix)

# orig_로 시작하는 이미지 불러오기
orig_images = load_images_from_folder(orig_images_folder, orig_images_prefix)

import random

def sample_images(orig_images, num_samples):
    """
    orig_images에서 num_samples 개수만큼 무작위로 이미지를 샘플링합니다.
    """
    if not isinstance(orig_images, list):
        orig_images = list(orig_images)  # 리스트로 변환

    if len(orig_images) < num_samples:
        raise ValueError("orig_images의 이미지 수가 num_samples보다 적습니다.")
    
    return random.sample(orig_images, num_samples)

def prepare_images_for_fid(images):
    """
    이미지를 FID 계산을 위해 준비합니다.
    - 이미지를 torch.Tensor로 변환
    - dtype을 torch.uint8로 설정
    """
    if isinstance(images, list):
        # 리스트를 텐서로 변환
        images = torch.stack([torch.tensor(img) for img in images])

    # 이미지를 0-255 범위로 변환
    if images.max() <= 1.0:
        images = (images * 255).byte()
    else:
        images = images.byte()

    return images
# vae_images의 이미지 개수
num_vae_images = len(vae_images)

# orig_images에서 vae_images의 개수만큼 무작위로 샘플링
sampled_orig_images = sample_images(orig_images, num_vae_images)

sampled_orig_images = prepare_images_for_fid(sampled_orig_images)

# FID 인스턴스 생성
fid = FrechetInceptionDistance(feature=64)

# 실제(real) 이미지로 orig_ 이미지 업데이트
fid.update(sampled_orig_images, real=True)

# 생성된(generated) 이미지로 vae_ 이미지 업데이트
fid.update(vae_images, real=False)

# FID 계산
fid_value = fid.compute()


/tmp/ipykernel_1636/2928651904.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.stack([torch.tensor(img) for img in images])
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:02<00:00, 38.9MB/s]


In [5]:
fid_value

tensor(0.4980)

In [6]:
def calculate_inception_score(images, batch_size=32, splits=10):
       # Inception 모델 로드
    inception_model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT, transform_input=False)
    inception_model.eval()

    # 이미지 변환 정의
    preprocess = transforms.Compose([
        transforms.ToPILImage(),        # 텐서를 PIL 이미지로 변환
        transforms.Resize((299, 299)),  # 이미지 크기 조정
        transforms.ToTensor(),          # 텐서로 변환
    ])

    # 이미지 텐서를 batch_size로 나누기
    n_images = images.size(0)
    preds = []
    for i in range(0, n_images, batch_size):
        batch = images[i:i + batch_size]
        batch = torch.stack([preprocess(img) for img in batch])  # 각 이미지를 전처리
        batch = batch.float() / 255.0  # 0~1 범위로 정규화
        with torch.no_grad():
            pred = inception_model(batch)
            preds.append(F.softmax(pred, dim=1).cpu())

    preds = torch.cat(preds, dim=0)

    # Inception Score 계산
    scores = []
    for i in range(splits):
        part = preds[i * (n_images // splits): (i + 1) * (n_images // splits), :]
        py = torch.mean(part, dim=0)
        scores.append(torch.exp(torch.mean(torch.sum(part * torch.log(part / py), dim=1))))

    return torch.mean(torch.tensor(scores)), torch.std(torch.tensor(scores))

In [7]:
import os
from PIL import Image
import torch
from torchvision import transforms, models
from torch.nn import functional as F

generated_images = load_images_from_folder(generate_images_folder, gen_image_prefix)

# Inception Score 계산
mean_score, std_score = calculate_inception_score(generated_images)

print(f"Inception Score: {mean_score.item()} ± {std_score.item()}")

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 154MB/s] 


Inception Score: 1.0000346899032593 ± 1.0199498319707345e-05
